In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

In [44]:
df = pd.read_csv('lego_Sets.csv')

In [45]:
df.list_price = df.list_price.astype(float)
df['review_difficulty'] = df['review_difficulty'].astype("category")
df['review_difficulty'] = df['review_difficulty'].cat.reorder_categories(['Very Easy',
                                                               'Easy',
                                                               'Average',
                                                               'Challenging',
                                                               'Very Challenging'],
                                                                        ordered = True)
df['review_difficulty'] = df['review_difficulty'].cat.codes
df.theme_name = df.theme_name.astype("category")
df.ages = df.ages.astype("category")


In [46]:
df1 = df.copy()
df1 = df1.drop(['prod_desc', 
                'prod_id', 
                'prod_long_desc', 
                'set_name',
                'theme_name',
                'country'], 
                 axis=1)

print(df1.shape)
# df2 = df1.copy().dropna()
df2 = pd.get_dummies(df1).dropna()
print(df2.shape)

(12261, 8)
(10466, 38)


In [5]:
X, Y = df2.drop(['list_price'], axis=1), df2['list_price']

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(
                                                    X, 
                                                    Y, 
                                                    test_size=0.3, 
                                                    random_state=361)

In [7]:




parametersGrid = {"max_iter": [1, 5, 10],
                  "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                  "l1_ratio": np.arange(0.0, 1.0, 0.1)}

eNet = ElasticNet()
grid = GridSearchCV(eNet, parametersGrid, scoring='r2', cv=5)
grid.fit(X_train, Y_train)
Y_pred = grid.predict(X_test)








lasso001 = Lasso(alpha=0.01, max_iter=10e5)
lasso001.fit(X_train,Y_train)
train_score001=lasso001.score(X_train,Y_train)
test_score001=lasso001.score(X_test,Y_test)
coeff_used001 = np.sum(lasso001.coef_!=0)
print ("training score for alpha=0.01:", train_score001 )
print ("test score for alpha =0.01: ", test_score001)
print ("number of features used: for alpha =0.01:", coeff_used001)

lasso_pred = lasso001.predict(X_test)

training score for alpha=0.01: 0.8610149222279182
test score for alpha =0.01:  0.8666229911149762
number of features used: for alpha =0.01: 33


In [8]:
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, lasso_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, lasso_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, lasso_pred)))

Mean Absolute Error: 16.689818748855455
Mean Squared Error: 925.5272718302724
Root Mean Squared Error: 30.422479712052933
